Alunos: Daniel de Paula, Gustavo Guerreiro e Mayara Cardoso Simões

# Trabalho Final de Aprendizado de Máquina sobre Visão Computacional: Classificação de Imagens de Cães e Gatos

O dataset utilizado é de propriedade da Microsoft e está disponível em: https://www.kaggle.com/datasets/shaunthesheep/microsoft-catsvsdogs-dataset

## Separação dos Dados em Treino, Teste e Validação

A primeira etapa da implementação é a de separação dos dados. Inicialmente o diretório se encontra no formato:
```
PetImages/
├── Cat
└── Dog
```

Como uma Rede Neural necessita de uma separação entre treino, teste e possivelmente validação, o dataset será reorganizado para seguir a seguinte estrutura mais comum nesse tipo de implementação:
```
dataset/
├── train/
│   ├── Cat/
│   └── Dog/
├── val/
│   ├── Cat/
│   └── Dog/
└── test/
    ├── Cat/
    └── Dog/
```
Para usar essa estrutura se utilizou a classe GeneratorBasedBuilder do TensorFlow para fazer a divisão mais eficiente e monstar a estrutura em treino, validação e teste.

## Importação das bibliotecas

In [5]:
from os import path
from glob import glob
from random import Random
from tensorflow import keras
from tensorflow.keras import layers, models
from PIL import Image, UnidentifiedImageError
from sklearn.model_selection import train_test_split
from tensorflow_datasets.core import GeneratorBasedBuilder, DatasetInfo, Version

import tensorflow as tf
import tensorflow_datasets as tfds

C:\Users\guto_\AppData\Local\Temp\ipykernel_31740\2960586207.py:9: DeprecationWarning: 'imghdr' is deprecated and slated for removal in Python 3.13
  import imghdr


## Definição da Classe do Dataset

Para organizar o dataset de uma forma mais eficiente foi criada a classe CatsDogs.
Inicialmente se tem uma função auxiliar chamada listar_imagens_validas, ela é usada para checar se a imagem sendo tratada é de fato um jpg válido ou foi corrompido.

Já a classe CatsDogs em si possui três métodos:
* _info: contém as informações contidas no dataset, no caso uma imagem de 3 dimensões (altura, largura e cor RGB) e o rótulo podendo ser "Cat" ou Dog.
* _split_generators: método principal que busca nas pastas as imagens dos gatos e cães, separa cada grupo em treino, validação e teste e então junta as imagens de cada animal.
* _generate_examples: usado para gerar os dados retornados em si, pegando cada imagem e atribuindo um id para ela.

In [6]:
def listar_imagens_validas(pasta, label):
    caminhos = glob(f"{pasta}/*")
    validos = []
    for caminho in caminhos:
        try:
            with Image.open(caminho) as img:
                formato = img.format
                img.verify()
                if formato == "JPEG":
                    validos.append((caminho, label))
        except (IOError, UnidentifiedImageError, SyntaxError):
            pass
    return validos


class CatsDogs(GeneratorBasedBuilder):
    VERSION = Version("1.0.0")
    SEED = 42
    PASTA_PADRAO = "PetImages"

    def _info(self):
        return DatasetInfo(
            builder=self,
            features=tfds.features.FeaturesDict({
                "image": tfds.features.Image(shape=(None, None, 3)),
                "label": tfds.features.ClassLabel(names=["Cat", "Dog"]),
            })
        )

    def _split_generators(self, dl_manager):
        raiz = self.PASTA_PADRAO

        gatos = listar_imagens_validas(f"{raiz}/Cat", 0)
        caes = listar_imagens_validas(f"{raiz}/Dog", 1)

        gatos_treino, gatos_resto = train_test_split(gatos, test_size=0.3, random_state=self.SEED)
        gatos_val, gatos_test = train_test_split(gatos_resto, test_size=0.5, random_state=self.SEED)

        caes_treino, caes_resto = train_test_split(caes, test_size=0.3, random_state=self.SEED)
        caes_val, caes_test = train_test_split(caes_resto, test_size=0.5, random_state=self.SEED)

        rng = Random(self.SEED)
        treino = gatos_treino + caes_treino
        rng.shuffle(treino)

        val = gatos_val + caes_val
        rng.shuffle(val)

        teste = gatos_test + caes_test
        rng.shuffle(teste)

        return {
            "train": self._generate_examples(treino),
            "val": self._generate_examples(val),
            "test": self._generate_examples(teste)
        }

    def _generate_examples(self, arquivos):
        for caminho, rotulo in arquivos:
            if not path.isfile(caminho):
                continue
            yield caminho, {"image": caminho, "label": rotulo}


## Instanciamento dos Datasets
A classe CatsDogs é instanciada e os datasets são construídos e carregados em variáveis.

In [38]:
builder = CatsDogs()
builder.download_and_prepare()

ds_train = builder.as_dataset(split="train")
ds_val = builder.as_dataset(split="val")
ds_test = builder.as_dataset(split="test")

## Pré-Processamento
As imagens são pré processadas, tendo inicialmente o seu tamanho ajustado e então os seus valores normalizados do formato RGB (0-255, 0-255, 0-255) para (0.0-1.0, 0.0-1.0, 0.0-1.0).

In [39]:
TAMANHO = 224

def preprocessamento(dicionario):
    image = dicionario["image"]
    label = dicionario["label"]
    image = tf.image.resize(image, (TAMANHO, TAMANHO))
    image = tf.cast(image, tf.float32) / 255.0

    return image, label

Além de aplicar o pré-processamento, os dados são organizados para serem divididos em batches para facilitar o processamento na rede e o prefetch para agilizar o processo de carregamento dos batches enquanto a rede é treinada.

In [40]:
BATCH = 32
ds_train = ds_train.map(preprocessamento).batch(BATCH).prefetch(tf.data.AUTOTUNE)
ds_val   = ds_val.map(preprocessamento).batch(BATCH).prefetch(tf.data.AUTOTUNE)
ds_test  = ds_test.map(preprocessamento).batch(BATCH).prefetch(tf.data.AUTOTUNE)

## Construção do Modelo
O modelo possui uma arquitetura sequencial, as etapas são as seguintes:
* Recebe um input de tamanho (128, 128, 3)
* Uma camada de data_augmentation que aplica transformações aleatórias nas imagens de treino: giros (RandomRotation), zoom (RandomZoom) e inversões horizontais (RandomFlip).
* Uma camada Convolucional com 32 filtros de formato 3x3.
* Max Pooling de janelas 2x2.
* Uma camada Convolucional com 64 filtros de formato 3x3.
* Max Pooling de janelas 2x2.
* Camadas são achatadas.
* Camada de Dropout 0.5 que desliga aleatoriamente 50% dos neurônios
* Camada densa inicial com 64 pontos de entrada.
* Camada final de saída com 2 valores de ativação possíveis (cão ou gato).

In [41]:
def criar_backbone(tamanho):
    backbone = keras.applications.EfficientNetB0(
        include_top=False,
        weights='imagenet',
        input_shape=(tamanho, tamanho, 3),
    )
    backbone.trainable = False
    return backbone

In [21]:
def criar_head(backbone):
    x = layers.GlobalAveragePooling2D()(backbone.output)
    x = layers.Dropout(0.2)(x)
    saida = layers.Dense(1, activation='sigmoid')(x)
    modelo = models.Model(inputs=backbone.input, outputs=saida)
    return modelo

In [22]:
def construir_modelo_transfer(tamanho):
    backbone = criar_backbone(tamanho)
    modelo = criar_head(backbone)

    modelo.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
        loss="binary_crossentropy",
        metrics=["accuracy"]
    )

    return modelo


In [23]:
modelo = construir_modelo_transfer(TAMANHO)
ds_train = ds_train.map(lambda x: (x["image"], x["label"]))
ds_val = ds_val.map(lambda x: (x["image"], x["label"]))
ds_test = ds_test.map(lambda x: (x["image"], x["label"]))

historico_1 = modelo.fit(
    ds_train,
    validation_data=ds_val,
    epochs=5
)


Epoch 1/5
542/542 ━━━━━━━━━━━━━━━━━━━━ 64s 109ms/step - accuracy: 0.4982 - loss: 0.7000 - val_accuracy: 0.4992 - val_loss: 0.7038
Epoch 2/5
542/542 ━━━━━━━━━━━━━━━━━━━━ 56s 103ms/step - accuracy: 0.5031 - loss: 0.6981 - val_accuracy: 0.4992 - val_loss: 0.7008
Epoch 3/5
542/542 ━━━━━━━━━━━━━━━━━━━━ 68s 126ms/step - accuracy: 0.5025 - loss: 0.6986 - val_accuracy: 0.4992 - val_loss: 0.7012
Epoch 4/5
542/542 ━━━━━━━━━━━━━━━━━━━━ 61s 112ms/step - accuracy: 0.5055 - loss: 0.6979 - val_accuracy: 0.4992 - val_loss: 0.7021
Epoch 5/5
542/542 ━━━━━━━━━━━━━━━━━━━━ 59s 108ms/step - accuracy: 0.5064 - loss: 0.6978 - val_accuracy: 0.4992 - val_loss: 0.7005


In [24]:
modelo.evaluate(ds_test)

116/116 ━━━━━━━━━━━━━━━━━━━━ 10s 87ms/step - accuracy: 0.4992 - loss: 0.7006


[0.7005689144134521, 0.4991918206214905]

In [29]:
data_augmentation = models.Sequential(
    [
        layers.RandomFlip("horizontal"),
        layers.RandomRotation(0.1),
        layers.RandomZoom(0.1),
    ],
    name="data_augmentation",
)

In [42]:
def construir_modelo():
    backbone = criar_backbone(TAMANHO)
    preprocess = tf.keras.applications.efficientnet.preprocess_input

    modelo = models.Sequential([
        layers.Input(shape=(TAMANHO, TAMANHO, 3)),

        data_augmentation,
        layers.Lambda(preprocess),
        backbone,
        layers.GlobalAveragePooling2D(),
        layers.Dropout(0.3),

        layers.Dense(2, activation='softmax')
    ])

    modelo.compile(
        optimizer='adam',
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )

    return modelo

## Treinamento
O modelo arquitetado é então rodado.

In [43]:
model = construir_modelo()
modelo = construir_modelo_transfer(TAMANHO)
ds_train = ds_train.map(lambda x: (x["image"], x["label"]))
ds_val = ds_val.map(lambda x: (x["image"], x["label"]))
ds_test = ds_test.map(lambda x: (x["image"], x["label"]))

historico = model.fit(
    ds_train,
    validation_data=ds_val,
    epochs=20
)

TypeError: in user code:


    TypeError: outer_factory.<locals>.inner_factory.<locals>.<lambda>() takes 1 positional argument but 2 were given


In [ ]:
import numpy as np
from sklearn.metrics import classification_report, accuracy_score

# 1. Extrair os rótulos verdadeiros (y_true) do dataset de teste
# Precisamos iterar no ds_test para pegar os lotes de rótulos e concatená-los
y_true = np.concatenate([y for x, y in ds_test], axis=0)

# 2. Gerar as previsões (y_pred) do modelo no dataset de teste
# O model.predict já processa todos os lotes
predictions = model.predict(ds_test)
y_pred = np.argmax(predictions, axis=1) # Pega o índice da maior probabilidade (0 ou 1)

# 3. Obter os nomes das classes (ex: ["Cat", "Dog"])
target_names = builder.info.features['label'].names

# 4. Imprimir o relatório de classificação
print("--- Relatório de Classificação ---")
print(classification_report(y_true, y_pred, target_names=target_names))

# Imprimir a acurácia geral separadamente (embora já esteja no relatório)
print(f"Acurácia Geral: {accuracy_score(y_true, y_pred):.4f}")

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

# 1. Calcular a matriz de confusão
cm = confusion_matrix(y_true, y_pred)

# 2. Visualizar a matriz
plt.figure(figsize=(7, 5))
sns.heatmap(cm,
            annot=True,
            fmt='d',
            cmap='Blues',
            xticklabels=target_names,
            yticklabels=target_names)

plt.xlabel('Rótulo Predito')
plt.ylabel('Rótulo Verdadeiro')
plt.title('Matriz de Confusão')
plt.show()